<a href="https://colab.research.google.com/github/Active-Galaxy/MiscVocab/blob/main/Ask_roBERTa_Anything.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Overview

This notebook uses the Haystack question answering system roBERTa to query a private set of documents. roBERTa, a large language model (LLM), built using a system called BERT, is not a conversational AI. It is more like a TV show's portrayal of AI from the '70s (maybe '60s). The input is not very conversational and will it won't handle standard database style questions.

In [ ]:
#@markdown Optionally (but highly recomended), an AI generated summary is available using Gemini, the Google LLM. When you <a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">get a GOOGLE_API_KEY</a> (they are free), check the box below before starting and insert the GOOGLE_API_KEY when prompted. This key is not stored.

connect_to_Gemini = True # @param {type:"boolean"}
GOOGLE_API_KEY = "XXXXXXXXXXXXXXXXX"
if connect_to_Gemini:
  GOOGLE_API_KEY =  input("Insert GOOGLE_API_KEY:  ").strip()
  if len(GOOGLE_API_KEY) != 39:
    print("Invalid GOOGLE_API_KEY -- click 'Runtime/Interupt execution' and restart")

from datetime import datetime
#print("Starting at", datetime.now().strftime('%H:%M:%S'))



## Starting the Notebook
To start the notebook, click on ***Runtime/Run all***. The notebook takes time to install and instantiate the software on the Google Colab server. Load time is about 3:45 for ***CPU only***. The spining animation and green checkmarks on the left hand side are an indication that something good is happening.

Note: Do not click on the black circles with the white triangles or you may have to restart everything.

Scroll to the bottom of the screen after starting the notebook.

In [ ]:
# @title Install Haystack
%%capture

!pip install --upgrade pip
print("----------------------------------------")
%pip install --root-user-action=ignore farm-haystack[inference,elasticsearch]
#!pip install --root-user-action=ignore farm-haystack[colab] # Requires restart

In [ ]:
# @title Connect to Elasticsearch
#%%capture

#!wget https://github.com/Active-Galaxy/MiscVocab/raw/main/Living%20by%20Faith.haystackDocs.txt -O docs.txt

#  https://docs.haystack.deepset.ai/docs/document_store
# Here, we're using the ElasticsearchDocumentStore which connects to a running Elasticsearch service.
# It's a fast and scalable text-focused storage option.
# This service runs independently from Haystack and persists even after the Haystack program has finished running.

%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [ ]:
# @title Start Elasticsearch Server
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
#@title Wait for server

import subprocess
def has_gpu():
  try:
    subprocess.check_output('nvidia-smi')
    print('GPU detected')
    return True
  except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('CPU Only')
    return False

wait30 = not has_gpu()
print("Waiting for server to start...")
import time

time.sleep(30)

import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

if wait30:
  print("Extra wait...")
  time.sleep(30)

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")
# ElasticsearchDocumentStore is up and running and ready to store the Documents.
print("Server started.")

In [ ]:
# @title Load Documents
%%capture
!wget https://github.com/Active-Galaxy/MiscVocab/raw/main/Living%20by%20Faith.haystackDocs.txt -O docs.txt

from haystack import Document
docs = []

doc_count = 0
found = False
with open('docs.txt', 'r') as file:
  for line in file:
    try:
      c,m = line.strip().split('\t')
    except:
      if not found:
        print("Read Error at line", doc_count, line)
        found = True
      continue

    d = Document(c, meta={'source': m}, id= "roBERTa"+str(doc_count))
    doc_count+=1
    docs.append(d)#'''

document_store.write_documents(docs)

In [ ]:
# @title Create querying_pipeline
%%capture

# https://docs.haystack.deepset.ai/docs/retriever
from haystack.nodes import BM25Retriever
retriever = BM25Retriever(document_store=document_store)

# https://huggingface.co/deepset/roberta-base-squad2
# https://docs.haystack.deepset.ai/docs/reader#models
# https://haystack.deepset.ai/tutorials/02_finetune_a_model_on_your_data

from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

from haystack import Pipeline
querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

model = None
GOOGLE_API_KEY = GOOGLE_API_KEY.strip()
if len(GOOGLE_API_KEY) == 39:
  !pip install -q -U --root-user-action=ignore google-generativeai
  import google.generativeai as genai
  try:
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
  except:
    pass

def queryDocs(query):
  prediction = querying_pipeline.run(query=query,
      params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 10}})

  return prediction

# Asking a Question
*While you are waiting for the server to start...*

When the software is ready for you, a long thin box will appear below along with the prompt "*Type query text or 'exit':*". Type your query into the box and hit ***enter***. The query does not have to be in the form of a question. A single word or phrase can also be used.

A query can show up to 5 responses from roBERTa. Each has a *score*, the extracted *answer*, the document *source*, and the *context* in which the answer was found. If you are using a GOOGLE_API_KEY, the Gemini LLM will return a summary based on the highest scoring document sections.

The input box will reappear after printing the current set of answers.

To start, try some of these queries:
- What is the first question?
- Who is Jesus Christ?
- What is the mystery of the gospel?
- What is the mystery of iniquity?
- How do I obtain perfection?
- I feel that I am trying too hard to find Jesus.
- I have trouble being humble

When finished with this notebook, click ***Runtime/Disconnect and delete runtime*** to be a good citizen.



In [ ]:
# @title Query loop (Keep Closed ❯)
# https://docs.haystack.deepset.ai/docs/optimization#choosing-the-right-top-k-values
# https://docs.haystack.deepset.ai/docs/pipelines#arguments
# https://haystack.deepset.ai/tutorials/24_building_chat_app
# https://docs.haystack.deepset.ai/docs/agent

# https://colab.research.google.com/github/deepset-ai/haystack-tutorials/blob/main/tutorials/22_Pipeline_with_PromptNode.ipynb#scrollTo=f6NFmpjEO-qb

print(doc_count, "document sections read.")

if model is None and connect_to_Gemini:
  print("Invalid GOOGLE_API_KEY")

from pprint import pprint
from haystack.utils import print_answers

from IPython.display import display, HTML

last_docs = ""
print("The query loop is available.")
while True:
  query = input("\nType query text or 'exit' then hit enter:\n").strip()
  if query == "exit":
      break

  prediction = queryDocs(query)

  max_score = 0
  for i in range(len(prediction["answers"])):
    score = prediction["answers"][i].score
    max_score = max(max_score, score)

  max_score = round(max_score, 2) * 100
  if max_score < 1:
    print("(No search results)")
    continue

  #print("Max score:", max_score, "%")
  current_docs = ""
  doc_ids = []
  for i in range(len(prediction["answers"])):
    answer = prediction["answers"][i]
    score = round(answer.score, 2) * 100
    if score < 1 :
      continue

    score = str(int(score)) + "%"
    simple = answer.answer.replace('\\', '')
    sents = simple.split(".")
    if len(sents) > 1:
      simple = sents[0] + "."
    elif not simple.endswith("."):
      simple += "."

    simple = simple[0].upper() + simple[1:]
    context = answer.context.replace('\\', '')
    if i < 5:
      print(score,"\t",simple,"\n\t\t", answer.meta, "\n\t\t","Context:",context)
      doc_ids.append(answer.document_ids[0])

    current_docs += context + "; "

  if model is not None:
    prompt = "Summarize the following related text for the given question\n\n"
    #docs = current_docs + "; " + last_docs
    last_docs = current_docs
    docs = ""
    for id in doc_ids:
      for document in  prediction["documents"]:
        if document.id == id:
          docs += document.content + "; "#'''

    prompt += '''Related text: ''' + docs + '''\n\n'''
    prompt += '''Question: ''' + query + '''\n\n'''
    #print(prompt)
    try:
      response = model.generate_content(prompt)
      display(to_markdown("**Summary**: "+response.text))
    except:
      display(to_markdown("**Summary**: (Gemini response error)"))
  print()
  #pprint(prediction)
  #print_answers(prediction, details="minimum")
